In [1]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd

from os.path import basename

In [2]:
path_ref = '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL'
path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-KERNEL-DECIMAL-'

In [3]:
import time
import pandas as pd
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML

# CSV file to save execution times
csv_filename = path_ref_name + "02-00-ExecutionTime.csv"

# Initialize the CSV file with header if it doesn't exist
try:
    df = pd.read_csv(csv_filename)
except FileNotFoundError:
    df = pd.DataFrame(columns=["cell_number", "execution_time"])
    df.to_csv(csv_filename, index=False)

# Function to log execution times
def log_execution_time(cell_number, execution_time):
    df = pd.read_csv(csv_filename)
    new_row = pd.DataFrame({"cell_number": [cell_number], "execution_time": [execution_time]})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv(csv_filename, index=False)

# Decorator for the magic function
@register_cell_magic
def log_time(line, cell):
    start_time = time.time()
    exec(cell, globals())
    end_time = time.time()
    execution_time = end_time - start_time

    # Current cell number
    cell_number = int(line) if line.isdigit() else None
    if cell_number is None:
        display(HTML("<b>Warning:</b> Cell number not provided!"))
        return

    # Log the execution time
    log_execution_time(cell_number, execution_time)
    display(HTML(f"<b>Cell {cell_number}:</b> Execution time = {execution_time:.6f} seconds"))

# To use the magic function in the notebook
# %%log_time 1
# cell code

In [4]:
%%log_time 1

# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

0    nameFileS:parameters.json
1    nameFileS:parameters.json
2    nameFileS:parameters.json
3    nameFileS:parameters.json
4    nameFileS:parameters.json
5    nameFileS:parameters.json
6    nameFileS:parameters.json
7    nameFileS:parameters.json
8    nameFileS:parameters.json
dtype: object
0    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/f...
1    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
2    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
3    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
4    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/s...
5    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/k...
6    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLID

/tmp/ipykernel_101500/3158088216.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


In [5]:
%%log_time 2

# .json file loop
df = pd.DataFrame(columns=['DEVICE','FUNCTION','DATA_TYPE',
                           'NUM_S','RANGE_S','NUM_SI','RANGE_SI',
                           'NUM_ARR','RANGE_ARR','NUM_MAT','RANGE_MAT'])  # 'BOARD',

listD = ["Arm","Atmega328p","Leon3","RiscV","Thumb"] # Insert the device/ISS you want to analyze
# listD = ["Bambu"]
listDT = ["int8_t","int16_t","int32_t","int64_t"] # Insert the data type you want to analyze
# listDT = ["float","double"]

In [6]:
%%log_time 3

def analyze_variables(strings):
    unique_strings = set()  # to keep track of unique strings without brackets 
    used_in_others = set()  # to keep track of strings used within brackets in other strings (Indexes)
    two_brackets = 0  # to count strings with exactly one pair of brackets (Arrays)
    four_brackets = 0  # to count strings with exactly two pairs of brackets (Matrixes)

    for string in strings:
        if '[' not in string and ']' not in string:
            # This is a unique string without brackets
            unique_strings.add(string)
        else:
            # This string contains brackets, find substrings inside brackets
            inside_brackets = string.split('[')[1:]
            inside_brackets = [s.split(']')[0] for s in inside_brackets]
            used_in_others.update(inside_brackets)

            # Count the number of brackets
            open_brackets = string.count('[')
            close_brackets = string.count(']')
            
            if open_brackets == 1 and close_brackets == 1:
                two_brackets += 1
            elif open_brackets == 2 and close_brackets == 2:
                four_brackets += 1

    # Find strings that appear only once and do not contain brackets
    unique_no_brackets = [s for s in unique_strings if strings.count(s) == 1 and s not in used_in_others] # (scalar value)

    return {
        'Scalars': len(unique_no_brackets),
        'Indexes': len(used_in_others),
        'Arrays': two_brackets,
        'Matrixes': four_brackets
    }

data_list = []

for i in range(0, len(pathFile)):
    pF = pathFile[i]
    print('pF:'+str(pF))

    with open(pF, 'r') as jf:
        data = jf.read()
    obj = json.loads(data)

    if os.name == 'nt':
        spR = rootFile[i].split('\\')
        print('spR:'+str(spR))
    else:
        spR = rootFile[i].split('/')
        print('spR:'+str(spR))
    
    function = spR[-1]
    print('function:'+str(function))
    # board = listD[i]
    # print('board:'+str(board))
    
    #Device loop
    deviceList = list(obj.keys())
    print('deviceList:'+str(deviceList))
    deviceListR = []
    for u in range(0, len(deviceList)):
        if deviceList[u] in listD:
            deviceListR.append(deviceList[u])
            print('deviceListR:'+str(deviceList[u]))

    for n in range(0, len(deviceListR)):
        # Data_type loop
        device = deviceListR[n]
        print('device:'+str(device))
        dtarr = list(obj[deviceListR[n]].keys())
        print('dtarr:'+str(dtarr))
        
        dtarrR = []
        for u in range(0, len(dtarr)):
            if dtarr[u] in listDT:
                dtarrR.append(dtarr[u])
                print(f'dtarr[{u}]:'+str(dtarr[u]))
        
        for j in range(0, len(dtarrR)):
            dataype = dtarrR[j]
            print('dataype:'+str(dtarrR[j]))
            lsubNP = (list(obj[deviceListR[n]][dtarrR[j]].keys()))
            print('lsubNP:'+str(lsubNP))
            lsubVP = (list(obj[deviceListR[n]][dtarrR[j]].values()))
            print('lsubVP:'+str(lsubVP))
            
            # sublist dimension
            ltot = len(lsubNP)
            print('ltot:'+str(ltot))
        
            # how many array are present
            na = 0
            nam = 0
            fl = 0
            flm = 0
            
            ark = 0
            arkm = 0
            
            for k in range(0, ltot): 
                #print(function, fl)
                if "[" in lsubNP[k] and (not "][" in lsubNP[k]):
                    na = na + 1
                    if fl == 0:
                        ark = k
                        fl = 1
                elif "[" in lsubNP[k] and "][" in lsubNP[k]:
                    nam = nam + 1
                    if flm == 0:
                        arkm = k
                        flm = 1

            '''
            # how many scalar index are present
            nsi = 0
            for k in range(0, ltot):
                flag = 0
                if not "[" in lsubNP[k]:
                    for keysl in lsubNP:
                        if (lsubNP[k] in keysl) and (not lsubNP[k] is keysl) and (flag == 0):          
                            flag = 1
                            nsi = nsi + 1           

            # how many scalar are present
            ns = ltot - na - nsi
            '''

            # check if is it negative
            ranArr = 0
            if na != 0:
                newstr = lsubVP[ark].replace("[", "")
                newstr = newstr.replace("]", "")
                newstr = newstr.split(",")[0]
                num = float(newstr)
                if num < 0:
                    ranArr = 1
                else:
                    ranArr = 0
                
            # check if is it negative
            ranMat = 0
            if nam != 0:
                newstrM = lsubVP[arkm].replace("[", "")
                newstrM = newstrM.replace("]", "")
                newstrM = newstrM.split(",")[0]
                numM = float(newstrM)
                if numM < 0:
                    ranMat = 1
                else:
                    ranMat = 0

            '''
            df = df.append({'DEVICE': device, 'FUNCTION': function, 'DATA_TYPE': dataype,
                            'NUM_S': ns, 'RANGE_S': 0,'NUM_SI': nsi,'RANGE_SI': 0,
                            'NUM_ARR': na,'RANGE_ARR': ranArr}, 
                            ignore_index=True)  # 'BOARD': board, 
            '''
            
            result = analyze_variables(lsubNP)
                
            data_list.append({'DEVICE': device, 'FUNCTION': function, 'DATA_TYPE': dataype,
                'NUM_S': result['Scalars'], 'RANGE_S': 0,'NUM_SI': result['Indexes'],'RANGE_SI': 0,
                'NUM_ARR': result['Arrays'],'RANGE_ARR': ranArr, 'NUM_MAT': result['Matrixes'],'RANGE_MAT': ranMat}) 
                # ignore_index=True)  # 'BOARD': board, 

df = pd.concat([pd.DataFrame([data]) for data in data_list], ignore_index=True)


pF:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/floydwarshall/parameters.json
spR:['..', '..', '..', 'SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL', 'SLIDE-x-RES', 'KERNEL_DECIMAL', 'floydwarshall']
function:floydwarshall
deviceList:['args', 'ALL', 'Arm', 'Thumb', '8051', 'Bambu', 'Atmega328p', 'Leon3', 'RiscV']
deviceListR:Arm
deviceListR:Thumb
deviceListR:Atmega328p
deviceListR:Leon3
deviceListR:RiscV
device:Arm
dtarr:['int8_t', 'uint8_t', 'int16_t', 'uint16_t', 'int32_t', 'uint32_t', 'int64_t', 'uint64_t', 'float', 'double']
dtarr[0]:int8_t
dtarr[2]:int16_t
dtarr[4]:int32_t
dtarr[6]:int64_t
dataype:int8_t
lsubNP:['size', 'a[size][size]']
lsubVP:['[2,32];100', '[-128,127]']
ltot:2
dataype:int16_t
lsubNP:['size', 'a[size][size]']
lsubVP:['[2,32];100', '[-32766,32766]']
ltot:2
dataype:int32_t
lsubNP:['size', 'a[size][size]']
lsubVP:['[2,32];100', '[-2147483640,2147483640]']
ltot:2
dataype:int64_t
lsubNP:['size', 'a[size][size]']
lsubVP:['[2,3

In [7]:
dfS = pd.read_csv("dfS.csv", sep=';')
dfSI = pd.read_csv("dfSI.csv", sep=';')

In [8]:
dfS = pd.read_csv("dfS_tot.csv", sep=';')
dfSI = pd.read_csv("dfSI_tot.csv", sep=';')

In [9]:
del dfS['Unnamed: 0']
del dfSI['Unnamed: 0']

In [10]:
columnsS = ['DEVICE','FUNCTION','DATA_TYPE','RANGE_S']
columnsSI = ['DEVICE','FUNCTION','DATA_TYPE','RANGE_SI']
dfS = dfS[columnsS]
dfSI = dfSI[columnsSI]

In [11]:
lent = len(dfS['RANGE_S'].tolist())
for i in range(0, lent):
    df.loc[(df['DEVICE'] == dfS['DEVICE'][i]) & 
           # (df['BOARD'] == dfS['BOARD'][i]) &
           (df['FUNCTION'] == dfS['FUNCTION'][i]) & 
           (df['DATA_TYPE'] == dfS['DATA_TYPE'][i]), 'RANGE_S'] = dfS['RANGE_S'][i]

In [12]:
lent = len(dfSI['RANGE_SI'].tolist())
for i in range(0, lent):
    df.loc[(df['DEVICE'] == dfSI['DEVICE'][i]) & 
           # (df['BOARD'] == dfSI['BOARD'][i]) & 
           (df['FUNCTION'] == dfSI['FUNCTION'][i]) & 
           (df['DATA_TYPE'] == dfSI['DATA_TYPE'][i]), 'RANGE_SI'] = dfSI['RANGE_SI'][i]

In [13]:
df.to_csv("RangeValueClassification_tot.csv", sep=';')